# Timer

In [1]:
from time import sleep
from datetime import datetime as dt

start_time = "29.12.2022, 23:20"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['STOCH', 'RSI']
# pattern = ['STOCH', 'RSI', 'LinearReg']
work_timeframe = '4h'
higher_timeframe = '1d'
opt_limit = 300
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'RSI': {'timeperiod': [14], 'low_bound': [20, 25, 30, 35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [15, 20, 25]},
                'LinearReg': {'timeperiod': [4, 6, 8, 10, 12], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 4h/1d, trade type is sell
Number of combinations is 12


  0%|                                                                                                                                                                                                   | 0/12 [00:00<?, ?it/s]

# Check local statistics

In [2]:
# 15m/1h
stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('e_ratio_rank', ascending=False).head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,LinearReg_timeperiod,LinearReg_low_bound,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank
57,STOCH_RSI_LinearReg,14,35,9,7,3,25,8,0,4.2060,9.2358,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,7.6693,7.6693,5.9970,5.7811,4.1207,3.3669,2.2461,1.9079,1.9079,1.8597,1.8142,1.8142,1.7662,1.7662,1.7662,-0.25,-0.53,-0.61,-0.93,-0.81,-0.82,-0.99,-0.68,-0.83,-0.67,-0.45,-0.40,-0.08,0.00,0.04,0.13,0.08,0.16,-0.06,0.01,0.15,0.02,-0.06,-0.27,6,5.620612,-0.327083,27.723675,-1.962500
56,STOCH_RSI_LinearReg,14,35,9,7,3,25,6,0,4.2060,9.2358,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,7.6693,7.6693,5.9970,5.7811,4.1207,3.3669,2.2461,1.9079,1.9079,1.8597,1.8142,1.8142,1.7662,1.7662,1.7662,-0.25,-0.53,-0.61,-0.93,-0.81,-0.82,-0.99,-0.68,-0.83,-0.67,-0.45,-0.40,-0.08,0.00,0.04,0.13,0.08,0.16,-0.06,0.01,0.15,0.02,-0.06,-0.27,6,5.620612,-0.327083,27.723675,-1.962500
55,STOCH_RSI_LinearReg,14,35,9,7,3,25,4,0,4.2060,9.2358,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,7.6693,7.6693,5.9970,5.7811,4.1207,3.3669,2.2461,1.9079,1.9079,1.8597,1.8142,1.8142,1.7662,1.7662,1.7662,-0.25,-0.53,-0.61,-0.93,-0.81,-0.82,-0.99,-0.68,-0.83,-0.67,-0.45,-0.40,-0.08,0.00,0.04,0.13,0.08,0.16,-0.06,0.01,0.15,0.02,-0.06,-0.27,6,5.620612,-0.327083,27.723675,-1.962500
58,STOCH_RSI_LinearReg,14,35,9,7,3,25,10,0,2.3211,4.9056,5.6363,4.2829,4.3040,4.7079,5.2128,4.9410,5.0508,4.3055,4.3055,3.8352,3.8103,3.1314,2.7495,2.0632,1.8211,1.8211,1.7508,1.7177,1.7288,1.6933,1.6933,1.6933,-0.17,-0.32,-0.41,-0.65,-0.63,-0.67,-0.79,-0.54,-0.79,-0.81,-0.62,-0.81,-0.16,0.00,0.00,0.00,0.00,0.00,0.00,-0.12,-0.17,-0.34,-0.50,-0.50,9,3.311767,-0.375000,20.805900,-3.375000
52,STOCH_RSI_LinearReg,14,35,9,7,3,20,8,0,6.9309,7.3945,9.2689,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,9.2371,4.8078,3.4997,2.2690,1.9306,1.9306,1.8608,1.7962,1.7962,1.7962,1.7962,1.7962,-0.36,-0.53,-0.97,-1.26,-0.95,-1.02,-1.32,-1.08,-1.02,-1.05,-0.81,-0.89,-0.57,-0.16,-0.08,0.08,0.08,0.16,0.48,0.05,-0.03,-0.19,-0.27,-0.67,4,6.171288,-0.515833,20.685150,-2.063333
51,STOCH_RSI_LinearReg,14,35,9,7,3,20,6,0,6.9309,7.3945,9.2689,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,9.2371,4.8078,3.4997,2.2690,1.9306,1.9306,1.8608,1.7962,1.7962,1.7962,1.7962,1.7962,-0.36,-0.53,-0.97,-1.26,-0.95,-1.02,-1.32,-1.08,-1.02,-1.05,-0.81,-0.89,-0.57,-0.16,-0.08,0.08,0.08,0.16,0.48,0.05,-0.03,-0.19,-0.27,-0.67,4,6.171288,-0.515833,20.685150,-2.063333
50,STOCH_RSI_LinearReg,14,35,9,7,3,20,4,0,6.9309,7.3945,9.2689,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,9.2371,4.8078,3.4997,2.2690,1.9306,1.9306,1.8608,1.7962,1.7962,1.7962,1.7962,1.7962,-0.36,-0.53,-0.97,-1.26,-0.95,-1.02,-1.32,-1.08,-1.02,-1.05,-0.81,-0.89,-0.57,-0.16,-0.08,0.08,0.08,0.16,0.48,0.05,-0.03,-0.19,-0.27,-0.67,4,6.171288,-0.515833,20.685150,-2.063333
48,STOCH_RSI_LinearReg,14,35,9,7,3,15,10,0,1.7561,1.9014,4.1168,4.9578,5.0339,6.4935,8.3181,8.5123,8.6676,8.9752,8.9752,8.9752,7.2908,6.1448,3.7635,2.1232,1.7442,1.7442,1.6013,1.5383,1.5383,1.5383,1.5383,1.5383,-0.15,-0.27,-0.80,-0.65,-0.72,-0.82,-1.26,-1.09,-1.13,-1.37,-1.07,-0.91,-0.74,-0.25,-0.16,-0.14,-0.00,0.08,0.57,0.18,0.16

In [7]:
# 15m/4h
stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('e_ratio_rank', ascending=False).head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,LinearReg_timeperiod,LinearReg_low_bound,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,e_ratio_avg,pct_price_diff_avg,forecast_rank,e_ratio_rank,price_rank
55,STOCH_RSI_LinearReg,14,35,9,7,3,25,4,0,0.9236,2.1793,2.4705,2.9869,3.0833,3.1879,3.4160,3.4391,3.4764,3.0617,2.9646,2.9520,3.1001,3.2845,3.1978,2.9134,2.5920,2.5334,2.4545,2.4256,2.4256,2.3451,2.3208,2.3481,-0.17,-0.26,-0.34,-0.40,-0.49,-0.59,-0.46,-0.45,-0.52,-0.16,-0.09,-0.13,-0.23,-0.10,-0.18,-0.71,-0.66,-0.56,-0.43,-0.28,-0.23,-0.33,-0.43,-0.43,11,2.753425,-0.359583,19.287675,19.287675,-3.955417
50,STOCH_RSI_LinearReg,14,35,9,7,3,20,4,0,0.7862,1.2180,1.5798,2.2421,2.3803,2.5089,2.8253,2.8679,2.9140,2.8058,2.6882,2.7379,2.9370,3.1850,3.0794,3.1804,2.9406,2.8401,2.7086,2.6614,2.6614,2.6614,2.6195,2.6609,-0.16,-0.26,-0.36,-0.43,-0.49,-0.65,-0.58,-0.54,-0.52,-0.16,-0.16,-0.16,-0.27,-0.17,-0.76,-0.85,-0.81,-0.62,-0.43,-0.47,-0.26,-0.34,-0.47,-0.50,9,2.570421,-0.434167,14.133788,14.133788,-3.907500
45,STOCH_RSI_LinearReg,14,35,9,7,3,15,4,0,0.7666,1.2955,1.7694,2.0182,2.2109,2.4403,2.9595,2.9376,3.0184,2.8262,2.6338,2.6338,2.6338,3.0365,3.3291,3.4974,3.3843,3.1786,2.9257,2.9257,2.9257,2.9257,2.8461,2.8461,-0.15,-0.24,-0.35,-0.41,-0.49,-0.62,-0.52,-0.52,-0.53,-0.16,-0.11,-0.15,-0.25,-0.13,-0.46,-0.78,-0.73,-0.59,-0.39,-0.37,-0.25,-0.33,-0.45,-0.47,6,2.665204,-0.393750,9.991225,9.991225,-2.362500
39,STOCH_RSI_LinearReg,14,30,9,7,3,20,12,0,0.9396,1.3919,1.4559,1.4712,1.5573,1.6302,1.8256,1.8035,1.5986,1.3733,1.3982,1.3946,1.4183,1.4282,1.4781,1.5065,1.5300,1.2701,1.2693,1.3336,1.3548,1.2161,1.0788,0.9752,-0.11,-0.27,-0.20,-0.17,-0.19,-0.37,-0.37,-0.26,-0.27,-0.26,-0.24,-0.23,-0.20,-0.21,-0.30,-0.41,-0.36,-0.27,-0.36,-0.46,-0.30,-0.35,-0.25,-0.09,20,1.404121,-0.270833,8.082417,8.082417,-5.416667
44,STOCH_RSI_LinearReg,14,30,9,7,3,25,12,0,0.9292,1.3446,1.3313,1.3507,1.4687,1.5558,1.7321,1.7144,1.5404,1.3409,1.3646,1.3573,1.3802,1.3846,1.4310,1.4580,1.4809,1.2212,1.2164,1.2647,1.2840,1.1548,1.0169,0.9277,-0.11,-0.25,-0.16,-0.10,-0.19,-0.36,-0.36,-0.26,-0.27,-0.26,-0.22,-0.21,-0.19,-0.21,-0.22,-0.38,-0.25,-0.17,-0.32,-0.45,-0.26,-0.29,-0.17,-0.06,22,1.343767,-0.238333,7.562867,7.562867,-5.243333
22,STOCH_RSI_LinearReg,14,25,9,7,3,20,8,0,0.5832,1.3065,1.4188,1.6702,1.9331,1.9331,2.4346,2.4346,2.7690,2.7690,2.7690,2.7690,2.7690,2.7690,2.7690,2.4788,2.4892,2.4035,2.4035,2.4035,2.4035,2.3548,2.2707,2.2707,-0.09,-0.51,-0.51,-0.57,-0.76,-0.73,-0.72,-0.69,-0.68,-0.62,-0.55,-0.55,-0.64,-0.63,-0.53,-0.44,-0.67,-0.72,-0.60,-0.71,-0.48,-0.35,-0.33,-0.43,4,2.273971,-0.562917,5.095883,5.095883,-2.251667
27,STOCH_RSI_LinearReg,14,25,9,7,3,25,8,0,0.5832,1.3065,1.4188,1.6702,1.9331,1.9331,2.4346,2.4346,2.7690,2.7690,2.7690,2.7690,2.7690,2.7690,2.7690,2.4788,2.4892,2.4035,2.4035,2.4035,2.4035,2.3548,2.2707,2.2707,-0.09,-0.51,-0.51,-0.57,-0.76,-0.73,-0.72,-0.69,-0.68,-0.62,-0.55,-0.55,-0.64,-0.63,-0.53,-0.44,-0.67,-0.72,-0.60,-0.71,-0.48,-0.35,-0.33,-0.43,4,2.273971,-0.562917,5.095883,5.095883,-2.251667
56,STOCH_RSI_LinearReg,14,35,9,7,3,25,6,0,0.8664,1.5018,1.5666,1.3417,1.3719,1.2373,1.2047,1.1598,1.1611,1.1564,1.1637,1.1107,1.1945,1.2634,1.2638,1.2343,1.1937,1.0891,0.9787,0.9672,0.9532,0.9447,0.8805,0.8821,-0.16,-0.20,-0.30,-0.23,-0.20,-0.11,-0.13,-0.12

In [3]:
# 4h/1d
stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('e_ratio_rank', ascending=False).head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,LinearReg_timeperiod,LinearReg_low_bound,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank
58,STOCH_RSI_LinearReg,14,35,9,7,3,25,10,0,0.8869,0.7709,0.6672,0.7839,1.0367,1.0548,1.0683,1.0804,1.1496,1.1957,1.1847,1.1210,1.0655,1.0655,1.0655,1.2117,1.6926,1.6926,1.8096,1.8421,1.9108,2.4737,2.4737,2.4737,-0.67,-0.47,-0.48,-0.92,-1.68,-1.10,-0.74,-0.85,-0.77,-0.58,0.18,0.00,-0.31,-0.47,-0.73,-1.37,-1.95,-1.54,-1.58,-2.11,-2.35,-4.55,-4.35,-3.93,6,1.365712,-1.388333,2.194275,-8.330000
53,STOCH_RSI_LinearReg,14,35,9,7,3,20,10,0,0.8869,0.7709,0.6672,0.7839,1.0367,1.0548,1.0683,1.0804,1.1496,1.1957,1.1847,1.1210,1.0655,1.0655,1.0655,1.2117,1.6926,1.6926,1.8096,1.8421,1.9108,2.4737,2.4737,2.4737,-0.67,-0.47,-0.48,-0.92,-1.68,-1.10,-0.74,-0.85,-0.77,-0.58,0.18,0.00,-0.31,-0.47,-0.73,-1.37,-1.95,-1.54,-1.58,-2.11,-2.35,-4.55,-4.35,-3.93,6,1.365712,-1.388333,2.194275,-8.330000
59,STOCH_RSI_LinearReg,14,35,9,7,3,25,12,0,1.7290,1.5608,1.1393,1.0245,1.2420,1.2920,1.2943,1.2956,1.3560,1.3963,1.3963,1.3034,1.3034,1.3197,1.1043,1.0469,1.3986,1.2435,1.2393,1.1898,1.1588,1.1719,1.1719,1.1719,-0.92,-0.97,-1.80,-0.99,-1.63,-1.77,-1.03,-0.71,-0.96,-0.80,-0.13,-0.13,-0.42,-0.74,-0.61,-1.77,-2.18,-2.12,-1.77,-2.18,-2.44,-2.44,-1.22,-0.88,7,1.272896,-1.275417,1.910271,-8.927917
54,STOCH_RSI_LinearReg,14,35,9,7,3,20,12,0,1.7290,1.5608,1.1393,1.0245,1.2420,1.2920,1.2943,1.2956,1.3560,1.3963,1.3963,1.3034,1.3034,1.3197,1.1043,1.0469,1.3986,1.2435,1.2393,1.1898,1.1588,1.1719,1.1719,1.1719,-0.92,-0.97,-1.80,-0.99,-1.63,-1.77,-1.03,-0.71,-0.96,-0.80,-0.13,-0.13,-0.42,-0.74,-0.61,-1.77,-2.18,-2.12,-1.77,-2.18,-2.44,-2.44,-1.22,-0.88,7,1.272896,-1.275417,1.910271,-8.927917
44,STOCH_RSI_LinearReg,14,30,9,7,3,25,12,0,4.0536,4.0536,1.1195,1.1195,1.2121,1.2276,1.2739,1.2739,1.2739,1.2739,1.2739,0.9944,0.9944,1.0482,1.0482,1.0482,2.0675,2.0675,2.0675,2.0675,2.1639,2.2483,2.2483,2.2483,-0.78,-0.74,-1.33,-1.21,-1.26,-1.35,-1.29,-0.91,-1.26,-0.76,-0.03,-0.28,-0.71,-1.44,-1.14,-1.48,-3.10,-2.92,-2.03,-2.50,-2.71,-2.55,-1.46,-1.18,2,1.727817,-1.434167,1.455633,-2.868333
39,STOCH_RSI_LinearReg,14,30,9,7,3,20,12,0,4.0536,4.0536,1.1195,1.1195,1.2121,1.2276,1.2739,1.2739,1.2739,1.2739,1.2739,0.9944,0.9944,1.0482,1.0482,1.0482,2.0675,2.0675,2.0675,2.0675,2.1639,2.2483,2.2483,2.2483,-0.78,-0.74,-1.33,-1.21,-1.26,-1.35,-1.29,-0.91,-1.26,-0.76,-0.03,-0.28,-0.71,-1.44,-1.14,-1.48,-3.10,-2.92,-2.03,-2.50,-2.71,-2.55,-1.46,-1.18,2,1.727817,-1.434167,1.455633,-2.868333
34,STOCH_RSI_LinearReg,14,30,9,7,3,15,12,0,4.0536,4.0536,1.1195,1.1195,1.2121,1.2276,1.2739,1.2739,1.2739,1.2739,1.2739,0.9944,0.9944,1.0482,1.0482,1.0482,2.0675,2.0675,2.0675,2.0675,2.1639,2.2483,2.2483,2.2483,-0.78,-0.74,-1.33,-1.21,-1.26,-1.35,-1.29,-0.91,-1.26,-0.76,-0.03,-0.28,-0.71,-1.44,-1.14,-1.48,-3.10,-2.92,-2.03,-2.50,-2.71,-2.55,-1.46,-1.18,2,1.727817,-1.434167,1.455633,-2.868333
52,STOCH_RSI_LinearReg,14,35,9,7,3,20,8,0,1.0293,1.0050,1.0050,1.0227,1.9329,2.0776,2.0776,2.1133,2.1133,1.7363,1.1648,1.0662,1.0662,1.0662,1.0662,1.0662,1.0662,1.0662,1.2601,1.2051,1.1844,1.6578,1.6578,1.6578,-0.71,-1.18,-0.79,-0.99,-2.97,-2.55,-2.90,-2.52,-2.10,-1.09,0.08,-0.67,-0.42,-1.01,-1.51,-1.43,-0.67,0.42,0.50,0.85,-0.28,-3.69,-3

In [ ]:
# 4h/1d no LR
stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('e_ratio_rank', ascending=False).head(20)

# Save new config data to config file

In [4]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '4h'
higher_timeframe = '1d'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'RSI': {'timeperiod': [14], 'low_bound': [35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [20]},
                'LinearReg': {'timeperiod': [12], 'low_bound': [0]}
              }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [3]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            9                  7                  3                  30                            83.541667   
                                                                                      25                            85.785833   
12             25            9                  7                  3                  30                            88.782083   
14             30            9                  7                  3                  25                            80.833333   
12             25            9                  7                  3                  25                            87.847917   
14             25            9                  7                  3                  30                            89.395000   
                                                                                      20                            89.395000   
                                                                                      25                            89.395000   
12             25            9                  7                  3                  20                            89.395000   
16             30            9                  7                  3                  20                            84.820833   
14             30            9                  7                  3                  30                            78.804583   
16             25            9                  7                  3                  20                            88.750000   
                                                                                      25                            88.750000   
                                                                                      30                            88.750000   
12             30            9                  7                  3                  30                            75.416667   
14             30            9                  7                  3                  20                            79.687500   
16             15            5                  4                  3                  25                            93.229167   
12             25            9                  7                  3                  15                            91.664583   
14             25            9                  7                  3                  15                            91.664583   
16             15            7                  4                  5                  25                            95.025694   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            9                  7                  3                  30                        -1.760000   
                                                                                      25                        -1.970833   
12             25            9                  7                  3                  30                        -2.084167   
14             30            9                  7                  3                  25                        -1.777917   
12             25            9                  7                  3                  25                        -2.421250   
14             25            9                  7                  3                  30                        -2.759583   
                                                                                      20                        -2.759583   
                                                                                      25                        -2.759583   
12            